In [10]:
import matplotlib.pyplot as plt
import numpy as np
path_subrotinas="/home/taniel/Desktop/Projetos_Deep/BCS/subrotinas.py"
exec(compile(open('param_LEA.py', "rb").read(), 'param_LEA.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open('subrotinas.py', "rb").read(), 'subrotinas.py', 'exec'))
exec(compile(open('exp_LEA.py', "rb").read(), 'exp_LEA.py', 'exec')) # Carrega função para leitura de dados experimentais
%matplotlib tk

0.24108276567583545 7.559321788388188 [-11.492505101438962, 204.43014465827954]
Dados carregados


Definindo as variáveis simbólicas

In [11]:
# Criando simbolica
from casadi import *
nx = 3; nu = 4;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena


In [12]:
pbh = x[0]
pwh = x[1]
q = x[2]
# =========================================================================
#  Define as entradas do BCS LEA   
# =========================================================================
#   freq = Cexp.referencia_frequencia_inversor(1:nsim)*0.1;                   % [Hz] frequencia de operacao
intervalo=np.array([int(0*3600),int(7.9*3600)])

#intervalo=np.array([1.8,3])
LEA=dados_LEA_Exp('Dados_BCSLEA_20210818.mat',intervalo)
pman=LEA['pressao_manifold_coriolis']*1e5
pres=LEA['pressao_reservatorio']*1e5
pm_0=pman[0]
pr_0=pres[0]
u_0=np.array([LEA['referencia_frequencia_inversor'][0],LEA['valvula_pneumatica_topo'][0],pm_0,pr_0])
x_0=np.array([LEA['pressao_fundo'][0]*1e5,LEA['pressao_choke'][0]*1e5,LEA['vazao'][0]/3600])
u_0
nsim=LEA['tempo']+1
ts=LEA['Ts']
tempo_hora = np.arange(0,nsim*ts,ts)/3600

0


In [13]:
fig3=plt.figure()
label = ['Pman','Pr','f','z'];
ax1=fig3.add_subplot(2,2,1)
ax1.plot(tempo_hora ,pman/1e5, label='Pman')
ax1.set_ylabel(label[0])
plt.grid(True)
ax2=fig3.add_subplot(2,2,2)
ax2.plot(tempo_hora,pres/1e5, label='Pr')
ax2.set_ylabel(label[1])
plt.grid(True)
ax3=fig3.add_subplot(2,2,3)
ax3.plot(tempo_hora,LEA['referencia_frequencia_inversor']*0.1, label='f')
ax3.set_ylabel(label[2])
plt.grid(True)
ax4=fig3.add_subplot(2,2,4)
ax4.plot(tempo_hora,LEA['valvula_pneumatica_topo'], label='z')
ax4.set_ylabel(label[3])
plt.grid(True)




In [14]:
# Entradas
# Valores máximos e mínimos para normalização
#Entradas - conforme binder e pavlov
#========================================
pbc=Lim_c(pbhlim)
pwc=Lim_c(pwhlim)
qc=Lim_c(qlim)
pbmin=pbhlim[0]
pwmin=pwhlim[0]
qmin=qlim[0]
#=============================
#Disabling normalization (some actions are necessary ahead to disable Hc, F1c, F2c, qcc)
# pbc=1;pwc=1;qc=1
# pbmin=0;pwmin=0;qmin=0
#=============================

fqref = u[0]; zcref = u[1]; pm=u[2]; pr=u[3]
#pm=2e6;
zc=zcref
fq=fqref

In [15]:
#=============================================
# Computing HEAD and pump pressure gain of LEA
q0 = (qc*q+qmin) / Cq * (f0 / fq)
H0 =  Head[0]*q0**4 +  Head[1]*q0**3 +  Head[2]*q0**2 + Head[3]*q0 + Head[4];
H = CH * H0 * (fq / f0) ** 2  # Head
Pp = rho * g * H  # Dp
#==============================================
# Electrical power and electrical current computing
P0 = -2.3599e9 * q0 ** 3 - 1.8082e7 * q0 ** 2 + 4.3346e6 * q0 + 9.4355e4
#P = Cp * P0 * (fq / f0) ** 3;  
#I = Inp * P / Pnp  
#==============================================
# Computing two volumes frictions in LEA piping
qan=q*qc+qmin # non normalized flow
Re_1 = (4*rho_1*qan)/(0.219*pi*mu);
Re_2 = (4*rho_2*qan)/(0.219*pi*mu);
fric_1=if_else(Re_1<4000, 64/Re_1,0.36*Re_1**(-0.25))
fric_2=if_else(Re_1<4000, 64/Re_2,0.36*Re_2**(-0.25))
F1 = (fric_1*qan**2*rho_1)/(2*pi*r1**3) #Frictional pressure drop below ESP
F2 = (fric_2*qan**2*rho_2)/(2*pi*r2**3) #Frictional pressure drop above ESP
#===========================================
#===========================================
# Computing intake pressure
pin = pbh*pbc+pbmin - rho * g * h1 - F1;
# Computing Reservoir flow
qr = PI * (pr - (pbh*pbc+pbmin));
# Computing flow across Choke valvule
qch = Cc * (zc / 100) * sqrt(fabs(pwh*pwc+pwmin - pm));
#============================================
# Nonlinear terms analysis
funcF1=Function('funcF1',[x],[F1])
funcF2=Function('funcF2',[x],[F2])
funcH=Function('funcH',[x,u],[H])
funcqch=Function('funqch',[x,u],[qch])
F1lim=(funcF1([0,0,qlim[0]]),funcF1([0,0,qlim[1]])) # Limits  F1
F2lim=(funcF2([0,0,qlim[0]]),funcF2([0,0,qlim[1]])) # Limits  F2
F1c=Lim_c(F1lim) # characteristic value F1
F2c=Lim_c(F2lim) # characteristic value F2
Hc=Lim_c(H_lim) # characteristic value H
qcc=Lim_c(qch_lim) # characteristic value qch

#=============================
#Disabling normalization (other actions is necessary ahead to disable pbc,pwc,qc)
# F1c=1;F1lim=(0,1) # characteristic value F1
# F2c=1;F2lim=(0,1) # characteristic value F2
# Hc=1;H_lim=(0,1) # characteristic value H
# qcc=1;qch_lim=(0,1) # characteristic value qch


#Normalizing nonlinear terms
##########################
qch=(qch-qch_lim[0])/qcc
F1=(F1-F1lim[0])/F1c
F2=(F2-F2lim[0])/F2c
H=(H-H_lim[0])/Hc
###########################

xss=x_0;uss=u_0

dpbhdt = (1/pbc)*b1/V1*(qr - (q*qc+qmin))
dpwhdt = (1/pwc)*b2/V2*((q*qc+qmin) - (qcc*qch+qch_lim[0]))
dqdt = (1/(qc*M))*(pbh*pbc+pbmin + rho * g * (H*Hc+H_lim[0]) - (pwh*pwc+pwmin) - rho*g*hw - (F1c*F1+F1lim[0]) - (F2c*F2+F2lim[0]))

dx=[dpbhdt,dpwhdt,dqdt]
print('Limites: pbh,pwh,q')
print(pbhlim,pwhlim,(qlim[0]*3600,qlim[1]*3600))
print('Limites: F1,F2,H,qch')
print(F1lim,F2lim,H_lim,qch_lim)

# print(u_0[3]/1e5,xss[0]/1e5)
# PI*(u_0[3]-xss[0])/1e5

Limites: pbh,pwh,q
(100000.0, 250000.0) (200000.0, 1100000.0) (0.1, 5.0)
Limites: F1,F2,H,qch
(DM(0.000439365), DM(0.000468628)) (DM(0.0110894), DM(0.011828)) [-11.492505101438962, 204.43014465827954] [6.69674349099543e-05, 0.0020998116078856078]


In [16]:
#xss = np.float32(np.array([8311024.82175957,2990109.06207437,0.00995042241351780]))

x0=np.array([pbmin,pwmin,qmin])#,0,0])
xc=np.array([pbc,pwc,qc])#,1,1])
xssn = (xss-x0)/xc
xssn

#xssn=xss
funcx1dot=Function('funcx1dot',[u,x],[dx[0]])
funcx2dot=Function('funcx2dot',[u,x],[dx[1]])
funcx3dot=Function('funcx3dot',[u,x],[dx[2]])
#dxout=[funcx1dot(uss,xssn), funcx2dot(uss,xssn),funcx3dot(uss,xssn)]
dxout=[funcx1dot(uss,xss), funcx2dot(uss,xss),funcx3dot(uss,xss)]
dxout


[DM(-2.81218e+07), DM(-83.1811), DM(-2.53029e+08)]

In [17]:
u_0[0]


40.0

In [18]:
# Função casadi
dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt) 
Eq_Estado = casadi.Function('Eq_Estado',[x,u],[dxdt],
                     ['x','u'],['dxdt'])

y=vertcat(pin,H);
ny = y.size1()
# Equações algébricas
sea_nl = casadi.Function('sea_nl',[x,u],[y,pin,H],\
                 ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u),
                   'sea_nl': sea_nl
                   }
}
#%% Calculo do estacionario
#% Func��o objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u']);
J = sum1(dxdt_0**2);
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;


In [19]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}
solver = nlpsol('solver', 'ipopt', MMQ, opt)
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],[sol['x']],['x0','uss'],['xss']);




In [20]:
BCS['Estacionario'] = Estacionario;
xss2=Estacionario(xssn,uss)
xss2



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



DM([0.778975, 0.451849, 0])

In [21]:
print(xss*xc+x0,uss)

[3.06844781e+10 4.58588934e+11 2.83303039e-05] [4.0000000e+01 2.0000000e+01 1.3393538e+04 2.1787500e+05]


In [22]:
xss

array([2.04562521e+05, 5.09543037e+05, 4.05937533e-04])

In [23]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = 2;      #% [pm]
tg = 2;      #% MV target
#% Parametros

#nsim=286
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x,u) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto p,ra integração da Eq_estado
opt = {'tf':ts,
       't0':0

       };   #% opções do integrador

In [24]:
int_odes = integrator('int_odes','cvodes',sedo,opt);
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv);


In [25]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);
Modelo_Predicao2 = Function('Modelo_Predicao2',[BCS['x'],BCS['u']],[res['xf']],['xk_1','uk_1'],['xk']);

In [26]:
xpk = xss;
uss
#xpk=normalizar(xss,xnorm)

#uk_1 = normaliza_u(uss[mv],unorm)
#uk_1=(uss[mv]-unorm[:,0])/unorm[:,1]
# MVS
# Aloca��o de variaveis
Xk = np.zeros((nx,1))

In [27]:
Yk = np.zeros((npv,1))
Uk = np.zeros((nmv,1))
Ymk = Yk
Ys = Yk
Ymin = Yk
Ymax = Yk

In [28]:
#Valores iniciais de simulação
tss=5 # Tempo inicial no estacionário
Uk= uss
# Povoar vetor de X com dados não normalizados



In [29]:
#======================================================
#Entradas aplicadas no LEA
val_pneumatica = LEA['valvula_pneumatica_topo']#[0:nsim]
frequencia = LEA['referencia_frequencia_inversor']#[0:nsim]
print(frequencia.shape,val_pneumatica.shape) 
#plt.plot(val_pneumatica)
#val_pneumatica
    

(285,) (285,)


In [30]:
nstep = nsim

a_range = [35,65]
b_range = [30,50] # periodo de retenção em cada amplitude

u_f,prbs=APRBS(a_range,b_range,nstep)
u_z,prbs=APRBS(a_range,b_range,nstep)

uk_1 = np.array([np.ones_like(u_z)*40,u_z]);
uk_1 = np.array([frequencia,val_pneumatica,pmam,pres]);
# uk_1 = np.array([np.ones_like(u_z)*40,np.ones_like(u_z)*20]);
# uk_1 = np.array([u_f,u_z]);


IndexError: index 285 is out of bounds for axis 0 with size 285

In [ ]:
#xpk = Modelo_Predicao2(xssn,uss) #valores normalizados
xpk = Modelo_Predicao2(xss,uss) #valores normalizados
Xk=xpk*xc+x0
# ypk=sea_nl(xpk,uss)[0]
# Yk=ypk*np.array([1,Hc])
# ypk[1]*Hc

In [ ]:
# xpk = Modelo_Predicao2(xss,uk_1[:,0])
# xi=(np.arange(0,int(nsim*ts),ts));
# #xi=np.arange(0,10,ts)
# xi.shape
# 
# uss
# Uk=uk_1[:,1]
# uk_1[:,2]

# print(xpk)
# k=1
# print(xss)
# k

In [ ]:
# print(xpk,uk_1[:,k])
# xpk = Modelo_Predicao2(xpk,uk_1[:,k])

# k+=1
# Xk = hcat([Xk,xpk])
# Uk = hcat([Uk,uk_1[:,k]])
# print(Xk)
# print(Uk)
# print(k)
print(Uk)
print(vcat([Xk[0,:],Xk[1,:],Xk[2,:]*3600]))



In [ ]:
print("Simulação iniciada")
teste=20
xi=tempo_hora[:teste]
#teste=nsim;xi=tempo_hora
#for k in range(1,5):
for k in range(1,teste):
    xpk = Modelo_Predicao2(xpk,uk_1[:,k])
    Xk = hcat([Xk,xpk*xc+x0]) #desnormalizar x e preencher vetor
    #ypk=sea_nl(xpk,uk_1[:,k])[0]
    #Yk = hcat([Yk,ypk*np.array([1,Hc])]);
    Uk = hcat([Uk,uk_1[:,k]])

print("Xk shape ="+str(Xk.shape))
# print("Uk shape ="+str(Uk.shape))
# print("Yk shape ="+str(Uk.shape))
# print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")
Xk[0,:].shape

In [ ]:
xi

In [ ]:
# fig1=plt.figure()
# label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
# for iy in range(0,npv):
#     ax = fig1.add_subplot(npv,1,iy+1)
#     #print(iy)
#     if iy == 0: # Pin
#         ax.plot(xi,(Yk[iy,:].T)/1e5, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(40,62))
#         plt.grid(True)
#     else: # H
#         ax.plot(xi,Yk[iy,:].T, label='Medição')
#         #ax.plot(xi,Ymk[iy,:].T,label='EKF')
#         ax.set_ylabel(label[iy])
#         ax.set(xlim=(xi[0], nsim*ts))
#        # ax.set(ylim=(580, 850))
#         plt.grid(True)
# #ax.plot(xi,Yk[2,:].T, label='EKF')
# ax.legend();
# ax.set_xlabel('Time (nT)')
# fig1.show()


In [ ]:
# #% Restrição
umin  = np.array([35, 0]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)'];
for iu in range(0,nmv):
    ax2=fig2.add_subplot(nmv,1,iu+1)
    ax2.plot(xi,Uk[iu,:].T, label='Medição')
    # ax2.plot([1,nsim*ts],[umin[iu], umin[iu]],'--r')
    # ax2.plot([1,nsim*ts],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax2.set_ylabel(label[iu])
    #ax2.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        ax2.set(ylim=(30, 70))

    plt.grid(True)


In [ ]:
# fig3=plt.figure()
# label = ['Pbh (bar)','Pwh (bar)','q(m3/s)'];
# for iu in range(0,3):
#     ax3=fig3.add_subplot(3,1,iu+1)
#     if iu==2:
#         ax3.plot(xi,(Xk[iu,:].T)*3600, label='Medição')
#         #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
#         #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
#         ax3.set_ylabel(label[iu])
#         #ax3.set(xlim=(xi[0], nsim*ts))
#         # if iu==0:
#         #     #ax2.set(ylim=(30, 70))
#         #     print(iu)
#         plt.grid(True)
#     else:
#         ax3.plot(xi,Xk[iu,:].T/1e5, label='Medição')
#         #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
#         #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
#         ax3.set_ylabel(label[iu])
#         #ax3.set(xlim=(xi[0], nsim*ts))
#         # if iu==0:
#         #     #ax2.set(ylim=(30, 70))
#         #     print(iu)
#         plt.grid(True)

fig3=plt.figure()
label = ['Pbh (bar)','Pwh (bar)','q(m3/s)'];
for iu in range(0,3):
    ax3=fig3.add_subplot(3,1,iu+1)
    if iu==2:
        ax3.plot(xi,(Xk[iu,:].T*3600), label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)
    else:
        ax3.plot(xi,Xk[iu,:].T/1e5, label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)


In [ ]:
# Xk[2,:]=Xk[2,:]*xc[2] #desnormalizar vazão
# Yk[1,:]=Yk[1,:]*Hc #desnormalizar Head

In [ ]:
exec(compile(open('envelope.py', "rb").read(), 'envelope.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
fig4,ax4=plt.subplots()
plt.grid(True)
BCS['Envelope']['fig'](ax4); # grafico do envelope
#
# Evolução dentro do envelope
ax4.plot(Xk[2,0:].T*3600,Yk[1,0:].T,'--k')
ax4.plot(Xk[2,0]*3600,Yk[1,0],'o')#,'MarkerFaceColor',[0,1,0],'MarkerEdgeColor',[0,0,0])
ax4.plot(Xk[2,-1]*3600,Yk[1,-1],'o')#,'MarkerFaceColor',[1,0,0],'MarkerEdgeColor',[0,0,0])
ax4.annotate('t=0',
             xy=(float(Xk[2,0]*3600),float(Yk[1,0])),
             xytext=(float(Xk[2,0]*3600)-5,float(Yk[1,0])+10),
             arrowprops=dict(facecolor='green', shrink=0.01))

ax4.annotate('t='+str(nsim),
             xy=(float(Xk[2,-1]*3600),float(Yk[1,-1])),
             xytext=(float(Xk[2,-1]*3600)-7,float(Yk[1,-1])+10),
             arrowprops=dict(facecolor='red', shrink=0.01))
plt.show()

np.savez('BCS_data_train_aprbs_zc.npz', t=xi, x1=Xk[0,:].T,x2=Xk[1,:].T,x3=Xk[2,:].T,zc=z_f)

In [ ]:
def ler_dados(str):
    with open(str, 'r') as f:
        d = f.readlines()
        epocas = np.zeros(len(d));
        tau = np.zeros(len(d));
        j = 0;
        data = []
        for i in d:
            k = i.rstrip().split(" ")
            data.append([float(i) if is_float(i) else i for i in k])
            epocas[j] = float(k[0])
            tau[j] = float(k[1])
            j += 1;
    
    return tau, epocas


In [ ]:
qcc/qc